In [176]:
# Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

In [177]:
# Load the training dataset
train_data = pd.read_csv('../data/SexLandmarks-train.csv')

In [178]:
# Apply KNN Imputer
imputer = KNNImputer(n_neighbors=9)
X_train = train_data.drop('sex', axis=1)
X_train_imputed = imputer.fit_transform(X_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)

In [179]:
# Calculate lengths and widths based on landmarks
w, h = 2240, 3200
euclidean_lengths = np.sqrt((X_train_imputed['x1'] * w - X_train_imputed['x9'] * w) ** 2 + (X_train_imputed['y1'] * h - X_train_imputed['y9'] * h) ** 2)
euclidean_widths = np.sqrt((X_train_imputed['x5'] * w - X_train_imputed['x15'] * w) ** 2 + (X_train_imputed['y5'] * h - X_train_imputed['y15'] * h) ** 2)

# Add new features to the DataFrame
X_train_imputed['euclidean_lengths'] = euclidean_lengths
X_train_imputed['euclidean_widths'] = euclidean_widths

# Calculate lengths and widths based on landmarks using Manhattan distance
manhattan_lengths = np.abs(X_train_imputed['x1'] * w - X_train_imputed['x9'] * w) + np.abs(X_train_imputed['y1'] * h - X_train_imputed['y9'] * h)
manhattan_widths = np.abs(X_train_imputed['x5'] * w - X_train_imputed['x15'] * w) + np.abs(X_train_imputed['y5'] * h - X_train_imputed['y15'] * h)

# Add new features to the DataFrame
X_train_imputed['manhattan_lengths'] = manhattan_lengths
X_train_imputed['manhattan_widths'] = manhattan_widths


In [180]:
# Calculate products, ratios, and differences for Euclidean and Manhattan distances
# Products

X_train_imputed['euclidean_area'] = X_train_imputed['euclidean_lengths'] * X_train_imputed['euclidean_widths']
X_train_imputed['manhattan_area'] = X_train_imputed['manhattan_lengths'] * X_train_imputed['manhattan_widths']

# Shoelace formula for area calculation based on 36 coordinates (landmarks)
X_train_imputed['footprint_area'] = 0.5 * np.abs(
    (X_train_imputed.filter(like='x') * np.roll(X_train_imputed.filter(like='y'), 1, axis=1)).sum(axis=1)
    - (X_train_imputed.filter(like='y') * np.roll(X_train_imputed.filter(like='x'), 1, axis=1)).sum(axis=1)
)


# Extract x and y coordinates for the top part (points 0 to 13)
x_top = X_train_imputed[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']].values
y_top = X_train_imputed[['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11', 'y12', 'y13']].values

# Apply the shoelace formula to calculate the area of the top part
top_area = 0.5 * np.abs(
    np.sum(x_top * np.roll(y_top, shift=1, axis=1), axis=1) -
    np.sum(y_top * np.roll(x_top, shift=1, axis=1), axis=1)
)

# Add the top part area as a new column to the dataframe
X_train_imputed['top_footprint_area'] = top_area

# Extract x and y coordinates for points 0 to 5 and point 13
x_circular = X_train_imputed[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x13']].values
y_circular = X_train_imputed[['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y13']].values

# Apply the shoelace formula to calculate the area for the top circular part
circular_area = 0.5 * np.abs(
    np.sum(x_circular * np.roll(y_circular, shift=1, axis=1), axis=1) -
    np.sum(y_circular * np.roll(x_circular, shift=1, axis=1), axis=1)
)

# Add the circular part area as a new column to the dataframe
X_train_imputed['circular_footprint_area'] = circular_area


# Function to calculate the centroid of a set of points
def calculate_centroid(x, y):
    centroid_x = np.mean(x)
    centroid_y = np.mean(y)
    return centroid_x, centroid_y

# Calculate the centroid for the circular points
centroid_x, centroid_y = calculate_centroid(x_circular.flatten(), y_circular.flatten())

# Calculate the radius for each point in the circular shape
radius = np.sqrt((x_circular - centroid_x)**2 + (y_circular - centroid_y)**2)

# Calculate the average radius
average_radius = np.mean(radius)

# Calculate the circumference
circumference = 2 * np.pi * average_radius

# Add the average radius and circumference to the dataframe
X_train_imputed['average_radius'] = average_radius
X_train_imputed['circular_circumference'] = circumference



# Ratios
X_train_imputed['euclidean_ratio'] = X_train_imputed['euclidean_lengths'] / (X_train_imputed['euclidean_widths'] + 1e-5)  # Add small value to avoid division by zero
X_train_imputed['manhattan_ratio'] = X_train_imputed['manhattan_lengths'] / (X_train_imputed['manhattan_widths'] + 1e-5)

# Differences
X_train_imputed['euclidean_difference'] = X_train_imputed['euclidean_lengths'] - X_train_imputed['euclidean_widths']
X_train_imputed['manhattan_difference'] = X_train_imputed['manhattan_lengths'] - X_train_imputed['manhattan_widths']

# Combined Euclidean-Manhattan measures
# Differences between Euclidean and Manhattan measures
X_train_imputed['length_diff_euclidean_manhattan'] = X_train_imputed['euclidean_lengths'] - X_train_imputed['manhattan_lengths']
X_train_imputed['width_diff_euclidean_manhattan'] = X_train_imputed['euclidean_widths'] - X_train_imputed['manhattan_widths']

# Sum of lengths and widths (Euclidean and Manhattan)
X_train_imputed['euclidean_sum'] = X_train_imputed['euclidean_lengths'] + X_train_imputed['euclidean_widths']
X_train_imputed['manhattan_sum'] = X_train_imputed['manhattan_lengths'] + X_train_imputed['manhattan_widths']


In [182]:
X_train_imputed

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,euclidean_difference,manhattan_difference,length_diff_euclidean_manhattan,width_diff_euclidean_manhattan,euclidean_sum,manhattan_sum,mean_distance,median_distance,min_distance,max_distance
0,0.556673,0.064617,0.459606,0.083429,0.395285,0.117782,0.361370,0.157043,0.328624,0.202848,...,1334.777651,1512.542017,-285.242653,-107.478287,2691.401317,3084.122257,0.723796,0.862558,0.0,1.837458
1,0.601897,0.171537,0.519308,0.152283,0.431714,0.167161,0.391671,0.213546,0.342868,0.280936,...,1294.217183,1408.191610,-119.553862,-5.579435,2735.009179,2860.142475,0.667644,0.840255,0.0,1.829963
2,0.327504,0.195426,0.437202,0.191268,0.523052,0.216216,0.575517,0.258836,0.610493,0.317048,...,1431.509706,1539.968203,-134.652068,-26.193571,3127.480153,3288.325792,0.813501,0.936506,0.0,1.478958
3,0.594496,0.111449,0.498477,0.120736,0.458640,0.153599,0.416760,0.191463,0.386116,0.235043,...,1157.001453,1378.319036,-303.755810,-82.438228,2346.522881,2732.716918,0.766604,0.857945,0.0,1.853600
4,0.616850,0.231525,0.544279,0.222445,0.489851,0.260579,0.450974,0.288725,0.410801,0.344109,...,1324.223624,1395.041420,-82.323704,-11.505908,2491.389959,2585.219571,0.722605,0.787337,0.0,1.776530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.391117,0.032064,0.458453,0.041082,0.507163,0.064128,0.560172,0.106212,0.603152,0.149299,...,1163.125302,1301.434273,-224.879981,-86.571010,2517.387890,2828.838881,0.858031,0.924981,0.0,1.885158
1996,0.622164,0.125962,0.519250,0.139049,0.463115,0.177492,0.415166,0.210209,0.374234,0.261739,...,1228.038446,1399.947086,-268.869242,-96.960602,2652.873463,3018.703307,0.721143,0.860857,0.0,1.854196
1997,0.594508,0.160830,0.496128,0.148230,0.443236,0.179358,0.407270,0.209004,0.379766,0.242356,...,1175.126458,1214.772883,-49.055637,-9.409212,2336.369943,2394.834793,0.653843,0.777829,0.0,1.789901
1998,0.551669,0.219335,0.402226,0.234927,0.316375,0.267152,0.262321,0.325364,0.244833,0.388773,...,1483.820344,1842.631772,-479.212915,-120.401487,3297.325796,3896.940198,0.973347,1.101290,0.0,2.028187


In [183]:
# Assuming 'sex' was originally in 'train_data'
target = train_data['sex'].values  # Target variable

# Split the data into training and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_imputed, target, test_size=0.2, random_state=42)

# Scale the combined features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val)  # Don't fit on validation set, only transform


In [184]:
# Initialize and train the Gradient Boosting model
gbm_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
gbm_model.fit(X_train_scaled, y_train_split)

# Validate the model
y_val_pred = gbm_model.predict(X_val_scaled)
print(classification_report(y_val, y_val_pred))
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       179
           1       0.85      0.85      0.85       221

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400

Validation Accuracy: 0.8325
